## Decode Sensor Data from GDB Dump File

In [ ]:
import re
import sys
import struct

def parse_gdb_memory_dump(filename):
    """
    Parse GDB memory dump and extract 64-bit values in correct order
    Format: address: data1 data2 (where data1 and data2 are consecutive 64-bit values)
    """
    memory_data = []
    
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        line = line.strip()
        if not line or line.startswith('//'):
            continue
            
        # Match pattern: address: hex_value hex_value
        # Example: 0x8000cb50:	0x727e8adbb377679f	0x61413a398304bf0c
        pattern = r'0x[0-9a-fA-F]+:\s+0x([0-9a-fA-F]+)\s+0x([0-9a-fA-F]+)'
        match = re.match(pattern, line)
        
        if match:
            # Extract both 64-bit values
            data1 = int(match.group(1), 16)  # First data value
            data2 = int(match.group(2), 16)  # Second data value
            
            memory_data.append(data1)
            memory_data.append(data2)
        else:
            # Handle single data value lines (like the last line in your file)
            single_pattern = r'0x[0-9a-fA-F]+:\s+0x([0-9a-fA-F]+)$'
            single_match = re.match(single_pattern, line)
            if single_match:
                data = int(single_match.group(1), 16)
                memory_data.append(data)
    
    return memory_data

def verify_parsing(filename, num_sensors=14):
    """
    Verify that parsing is working correctly by showing first few entries
    """
    memory_data = parse_gdb_memory_dump(filename)
    
    print("Verification of parsing:")
    print("=" * 50)
    
    # Show first few entries with their expected addresses
    base_addr = 0x8000A000
    for i in range(min(10, len(memory_data))):
        expected_addr = base_addr + (i * 8)
        print(f"Index {i:2d}: 0x{memory_data[i]:016X} (Expected addr: 0x{expected_addr:08X})")
    
    # Check if data length makes sense
    print(f"\nTotal data points: {len(memory_data)}")
    if len(memory_data) % num_sensors == 0:
        num_iterations = len(memory_data) // num_sensors
        print(f"Complete iterations: {num_iterations}")
    else:
        print(f"Warning: {len(memory_data)} data points not divisible by {num_sensors} sensors")
    
    return memory_data

def decode_sensor_data(data_64bit):
    """
    Decode 64-bit sensor data into individual components
    """
    # Extract each field according to the bit layout
    thermal = data_64bit & 0x3FF  # Bits 9-0 (10 bits)
    voltage = (data_64bit >> 10) & 0x3FF  # Bits 19-10 (10 bits)
    current = (data_64bit >> 20) & 0xFFFF  # Bits 35-20 (16 bits)
    power = (data_64bit >> 36) & 0x3FFFFFF  # Bits 61-36 (26 bits)
    unused = (data_64bit >> 62) & 0x3  # Bits 63-62 (2 bits)
    
    return {
        'thermal': thermal,
        'voltage': voltage,
        'current': current,
        'power': power,
        'unused': unused
    }

def process_sensor_memory_dump(filename, num_sensors=14, base_addr=0x8000A000):
    """
    Process the complete memory dump and organize by sensor and iteration
    """
    memory_data = parse_gdb_memory_dump(filename)
    
    if not memory_data:
        print("No valid data found in the file. Please check the format.")
        return
    
    print(f"Found {len(memory_data)} 64-bit values in memory dump")
    
    # Calculate number of iterations
    if len(memory_data) % num_sensors != 0:
        print(f"Warning: Data length ({len(memory_data)}) is not divisible by number of sensors ({num_sensors})")
    
    num_iterations = len(memory_data) // num_sensors
    print(f"Number of complete iterations: {num_iterations}")
    
    # Process data by iterations and sensors
    results = []
    
    for iteration in range(num_iterations):
        iteration_data = []
        print(f"\n=== Iteration {iteration + 1} ===")
        
        for sensor in range(num_sensors):
            data_index = iteration * num_sensors + sensor
            if data_index < len(memory_data):
                raw_data = memory_data[data_index]
                decoded = decode_sensor_data(raw_data)

                # assert decoded['unused'] == 0, f"Unused bits are not zero for sensor {sensor} in iteration {iteration + 1}, top 2-bits are: {decoded['unused']}"

                # Calculate memory address
                addr = base_addr + (data_index * 8)  # 8 bytes per 64-bit value
                
                sensor_info = {
                    'iteration': iteration + 1,
                    'sensor': sensor,
                    'address': addr,
                    'raw_data': raw_data,
                    **decoded
                }
                
                iteration_data.append(sensor_info)
                
                print(f"Sensor {sensor:2d} @ 0x{addr:08X}: "
                      f"Raw=0x{raw_data:016X} | "
                      f"T={decoded['thermal']:4d} | "
                      f"V={decoded['voltage']:4d} | "
                      f"I={decoded['current']:5d} | "
                      f"P={decoded['power']:8d}")
        
        results.append(iteration_data)
    
    return results

def save_decoded_data(results, output_filename):
    """
    Save decoded data to a CSV file for further analysis
    """
    with open(output_filename, 'w') as f:
        # Write header
        f.write("Iteration,Sensor,Address,Raw_Data_Hex,Thermal,Voltage,Current,Power\n")
        
        # Write data
        for iteration_data in results:
            for sensor_info in iteration_data:
                f.write(f"{sensor_info['iteration']},"
                       f"{sensor_info['sensor']},"
                       f"0x{sensor_info['address']:08X},"
                       f"0x{sensor_info['raw_data']:016X},"
                       f"{sensor_info['thermal']},"
                    #    f"{sensor_info['voltage'] if sensor_info['voltage'] > 600 else sensor_info['voltage'] + 1024},"
                        f"{sensor_info['voltage']},"
                       f"{sensor_info['current']},"
                       f"{sensor_info['power']}\n")
    
    print(f"\nDecoded data saved to {output_filename}")


In [ ]:
# Configuration
input_filename = "/home/bwoah/tools/C_compile_template/results/log.txt"
csv_filename = "/home/bwoah/tools/C_compile_template/results/decoded_sensor_data.csv"
num_sensors = 14
base_addr = 0x8000A000

print("Sensor Data Decoder")
print("=" * 50)
print(f"Input file: {input_filename}")
print(f"Number of sensors: {num_sensors}")
print(f"Base address: 0x{base_addr:08X}")

try:
    # First verify parsing
    memory_data = verify_parsing(input_filename, num_sensors)
    if not memory_data:
        print("No valid data found in the file. Please check the format.")
        sys.exit(1)

    results = process_sensor_memory_dump(input_filename, num_sensors, base_addr)
    
    if results:
        # Save to CSV file
        save_decoded_data(results, csv_filename)

        # Print summary
        print(f"\nSummary:")
        print(f"Total iterations processed: {len(results)}")
        print(f"Total sensor readings: {len(results) * num_sensors}")
        
except FileNotFoundError:
    print(f"Error: Could not find file '{input_filename}'")
    print("Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"Error processing file: {e}")


## Plot sensor data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def load_and_plot_sensor_data(csv_filename, sensor_id=13, window_size=20):
    """
    Load sensor data from CSV and plot temperature and voltage for a specific sensor
    """
    # Load the CSV data
    df = pd.read_csv(csv_filename)
    
    print(f"Loaded data shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Iterations range: {df['Iteration'].min()} to {df['Iteration'].max()}")
    print(f"Sensors range: {df['Sensor'].min()} to {df['Sensor'].max()}")
    
    # Filter data for the specific sensor
    sensor_data = df[df['Sensor'] == sensor_id].copy()
    
    if sensor_data.empty:
        print(f"No data found for sensor {sensor_id}")
        return
    
    print(f"\nSensor {sensor_id} data points: {len(sensor_data)}")
    
    # Sort by iteration to ensure proper plotting order
    sensor_data = sensor_data.sort_values('Iteration')
    
    # Calculate moving averages
    sensor_data['temp_moving_avg'] = sensor_data['Thermal'].rolling(window=window_size, center=True).mean()
    sensor_data['volt_moving_avg'] = sensor_data['Voltage'].rolling(window=window_size, center=True).mean()

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    fig.suptitle(f'Sensor {sensor_id} - Temperature and Voltage Over Time (Moving Avg: {window_size} points)', fontsize=16)
    
    # Plot Temperature - Raw data emphasized, moving average lighter
    ax1.plot(sensor_data['Iteration'], sensor_data['Thermal'], 'r-', linewidth=2, label='Raw Temperature')
    ax1.plot(sensor_data['Iteration'], sensor_data['temp_moving_avg'], 'r-', linewidth=1.5, alpha=0.6, label=f'Moving Average ({window_size})')
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Temperature (10-bit code)')
    ax1.set_title('Temperature Readings')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(sensor_data['Iteration'].min(), sensor_data['Iteration'].max())
    ax1.legend()
    
    # Plot Voltage - Raw data emphasized, moving average lighter
    ax2.plot(sensor_data['Iteration'], sensor_data['Voltage'], 'b-', linewidth=2, label='Raw Voltage')
    ax2.plot(sensor_data['Iteration'], sensor_data['volt_moving_avg'], 'b-', linewidth=1.5, alpha=0.6, label=f'Moving Average ({window_size})')
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Voltage (10-bit code)')
    ax2.set_title('Voltage Readings')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(sensor_data['Iteration'].min(), sensor_data['Iteration'].max())
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics for the moving averages
    temp_ma_mean = sensor_data['temp_moving_avg'].mean()
    temp_ma_std = sensor_data['temp_moving_avg'].std()
    volt_ma_mean = sensor_data['volt_moving_avg'].mean()
    volt_ma_std = sensor_data['volt_moving_avg'].std()
    
    print(f"\n=== Sensor {sensor_id} Moving Average Statistics (Window: {window_size}) ===")
    print(f"Temperature MA: Mean={temp_ma_mean:.2f}, Std={temp_ma_std:.2f}")
    print(f"Voltage MA:     Mean={volt_ma_mean:.2f}, Std={volt_ma_std:.2f}")
    
    return sensor_data

def plot_temperature_range(csv_filename, sensor_id=13, start_iter=400, end_iter=600, window_size=10):
    """
    Plot temperature variation for a specific sensor within a given iteration range with moving average
    """
    # Load the CSV data
    df = pd.read_csv(csv_filename)
    
    # Filter data for the specific sensor
    sensor_data = df[df['Sensor'] == sensor_id].copy()
    
    if sensor_data.empty:
        print(f"No data found for sensor {sensor_id}")
        return
    
    # Filter for the specified iteration range
    range_data = sensor_data[
        (sensor_data['Iteration'] >= start_iter) & 
        (sensor_data['Iteration'] <= end_iter)
    ].copy()
    
    if range_data.empty:
        print(f"No data found for sensor {sensor_id} between iterations {start_iter}-{end_iter}")
        print(f"Available iteration range: {sensor_data['Iteration'].min()} to {sensor_data['Iteration'].max()}")
        return
    
    # Sort by iteration
    range_data = range_data.sort_values('Iteration')
    
    # Calculate moving average
    range_data['temp_moving_avg'] = range_data['Thermal'].rolling(window=window_size, center=True).mean()
    
    print(f"Sensor {sensor_id} data points in range {start_iter}-{end_iter}: {len(range_data)}")
    
    # Create the plot
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    fig.suptitle(f'Sensor {sensor_id} - Temperature Variation (Iterations {start_iter}-{end_iter})', fontsize=16)
    
    # Plot Temperature - Raw data emphasized, moving average lighter
    ax.plot(range_data['Iteration'], range_data['Thermal'], 'r-', linewidth=2, label='Raw Temperature')
    ax.plot(range_data['Iteration'], range_data['temp_moving_avg'], 'r-', linewidth=1.5, alpha=0.6, label=f'Moving Average ({window_size})')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Temperature (10-bit code)')
    ax.set_title('Temperature Readings with Moving Average')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(start_iter, end_iter)
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed statistics
    temp_min = range_data['Thermal'].min()
    temp_max = range_data['Thermal'].max()
    ma_min = range_data['temp_moving_avg'].min()
    ma_max = range_data['temp_moving_avg'].max()
    
    print(f"\n=== Sensor {sensor_id} Temperature Statistics (Iterations {start_iter}-{end_iter}) ===")
    print(f"Raw Data - Min: {temp_min}, Max: {temp_max}, Range: {temp_max - temp_min}")
    print(f"Moving Avg - Min: {ma_min:.2f}, Max: {ma_max:.2f}, Range: {ma_max - ma_min:.2f}")
    
    # Calculate temperature change rate for moving average
    if len(range_data) > 1:
        # Use moving average for trend analysis
        valid_ma = range_data['temp_moving_avg'].dropna()
        if len(valid_ma) > 1:
            temp_change = valid_ma.iloc[-1] - valid_ma.iloc[0]
            iter_span = len(valid_ma) - 1
            change_rate = temp_change / iter_span if iter_span > 0 else 0
            print(f"Moving Average Trend: {temp_change:.2f} change over {iter_span} points")
            print(f"Average rate: {change_rate:.4f} units/iteration")
    
    return range_data

def plot_all_sensors_overview(csv_filename, window_size=15):
    """
    Plot an overview of all sensors' temperature and voltage with both raw data and moving averages
    """
    df = pd.read_csv(csv_filename)
    
    # Calculate moving averages for all sensors
    sensor_list = df['Sensor'].unique()
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(sensor_list)))
    
    # Plot all sensors' temperature - raw data and moving averages
    for i, sensor in enumerate(sensor_list):
        sensor_data = df[df['Sensor'] == sensor].sort_values('Iteration')
        temp_ma = sensor_data['Thermal'].rolling(window=window_size, center=True).mean()
        
        # Raw data - emphasized
        ax1.plot(sensor_data['Iteration'], sensor_data['Thermal'], 
                color=colors[i], linewidth=1.5, label=f'Sensor {sensor} (Raw)', alpha=0.8)
        # Moving average - lighter
        ax1.plot(sensor_data['Iteration'], temp_ma, 
                color=colors[i], linewidth=1, linestyle='--', alpha=0.5, 
                label=f'Sensor {sensor} (MA-{window_size})')
    
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Temperature (10-bit code)')
    ax1.set_title(f'Temperature Readings - All Sensors (Raw + Moving Average)')
    ax1.grid(True, alpha=0.3)
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    
    # Plot all sensors' voltage - raw data and moving averages
    for i, sensor in enumerate(sensor_list):
        sensor_data = df[df['Sensor'] == sensor].sort_values('Iteration')
        volt_ma = sensor_data['Voltage'].rolling(window=window_size, center=True).mean()
        
        # Raw data - emphasized
        ax2.plot(sensor_data['Iteration'], sensor_data['Voltage'], 
                color=colors[i], linewidth=1.5, label=f'Sensor {sensor} (Raw)', alpha=0.8)
        # Moving average - lighter
        ax2.plot(sensor_data['Iteration'], volt_ma, 
                color=colors[i], linewidth=1, linestyle='--', alpha=0.5, 
                label=f'Sensor {sensor} (MA-{window_size})')
    
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Voltage (10-bit code)')
    ax2.set_title(f'Voltage Readings - All Sensors (Raw + Moving Average)')
    ax2.grid(True, alpha=0.3)
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    
    plt.tight_layout()
    plt.show()

def compare_sensors(csv_filename, sensor_list=[0, 7, 13], window_size=15):
    """
    Compare temperature and voltage for selected sensors with both raw data and moving averages
    """
    df = pd.read_csv(csv_filename)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(sensor_list)))
    
    for i, sensor_id in enumerate(sensor_list):
        sensor_data = df[df['Sensor'] == sensor_id].sort_values('Iteration')
        
        # Calculate moving averages
        temp_ma = sensor_data['Thermal'].rolling(window=window_size, center=True).mean()
        volt_ma = sensor_data['Voltage'].rolling(window=window_size, center=True).mean()
        
        # Temperature plot - raw data emphasized
        ax1.plot(sensor_data['Iteration'], sensor_data['Thermal'], 
                color=colors[i], linewidth=2, label=f'Sensor {sensor_id} (Raw)', alpha=0.8)
        ax1.plot(sensor_data['Iteration'], temp_ma, 
                color=colors[i], linewidth=1.5, linestyle='--', alpha=0.5, 
                label=f'Sensor {sensor_id} (MA-{window_size})')
        
        # Voltage plot - raw data emphasized
        ax2.plot(sensor_data['Iteration'], sensor_data['Voltage'], 
                color=colors[i], linewidth=2, label=f'Sensor {sensor_id} (Raw)', alpha=0.8)
        ax2.plot(sensor_data['Iteration'], volt_ma, 
                color=colors[i], linewidth=1.5, linestyle='--', alpha=0.5, 
                label=f'Sensor {sensor_id} (MA-{window_size})')
    
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Temperature (10-bit code)')
    ax1.set_title(f'Temperature Comparison (Raw + Moving Average)')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Voltage (10-bit code)')
    ax2.set_title(f'Voltage Comparison (Raw + Moving Average)')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot specific sensor (sensor 13)
sensor_data = load_and_plot_sensor_data(csv_filename, sensor_id=13, window_size=20)

# Plot temperature variation between iterations 400-600 for sensor 13
temp_range_data = plot_temperature_range(csv_filename, sensor_id=13, start_iter=300, end_iter=500, window_size=5)

# Plot overview of all sensors
plot_all_sensors_overview(csv_filename)

# Compare specific sensors
compare_sensors(csv_filename, sensor_list=[0, 7, 13], window_size=20)